In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode
import pandas_profiling
from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,KFold
from sklearn.metrics import classification_report,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.metrics import auc
from statsmodels.stats.outliers_influence import variance_inflation_factor

import os
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_colwidth',500)
pd.set_option('display.max_columns',5000)

encoder = LabelEncoder()

C:\Users\Trinadh\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
C:\Users\Trinadh\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Trinadh\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Trinadh\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Trinadh\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, g

In [5]:
df = pd.read_csv('final_train.csv')
df.head()

,Unnamed: 0,coupon_id,customer_id,redemption_status,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum
0,0,27,1053,0,1105,0,6,-5349.93,-1.241694,191.0,125,19.224000,-177447.090167,-44.367382,436274.446738,4661.0,1.169768,117.276864,3713.0,-45.609077,-182797.020167,0,47,3.0,1.0,0.0,1.0,0.0,5.0,-0.287258,1,11,2,208,5,-25.583099,340.487097,163.966826,-89.05,310,-7930.760842,105551,50829.715972
1,1,116,48,0,56,1,6,0.00,0.000000,0.0,3,20.333333,-586.140000,-10.207037,5323.510000,86.0,1.050926,70.885046,75.0,-10.207037,-586.140000,0,47,2.0,0.0,0.0,2.0,0.0,3.0,-3.215039,12,4,5,244,6,-20.013685,31.540260,188.638474,-1237.79,385,-7705.268702,12143,72625.812434
2,2,635,205,0,560,0,11,-605.54,-2.987321,15.0,67,1.761194,-12475.083333,-76.767366,48980.998333,202.0,1.353802,342.826195,142.0,-79.754687,-13080.623333,1,32,3.0,0.0,0.0,2.0,0.0,7.0,-2.212082,85,27,2,533,3,-12.873370,1.392784,112.067666,-2145.72,970,-12487.169143,1351,108705.636349
3,3,644,1050,0,611,0,6,-17.81,-4.452500,1.0,4,3.500000,-260.750000,-4.451964,3697.783333,47.0,1.110390,88.694903,39.0,-8.904464,-278.560000,0,47,3.0,0.0,0.0,2.0,0.0,5.0,-0.751477,2,9,4,216,8,-12.880868,1.291139,100.896997,-178.10,237,-3052.765833,306,23912.588333
4,4,1017,1489,0,1558,0,6,-569.92,-0.198055,23.0,32,27.312500,-46796.046667,-30.665116,289181.848333,1610.0,1.071044,200.991719,1420.0,-30.863170,-47365.966667,0,48,3.0,0.0,0.0,2.0,0.0,3.0,-0.471548,10,2,4,327,5,-12.264174,247.443060,85.016352,-265.01,562,-6892.466021,139063,47779.189609


In [6]:
df.drop(columns=['Unnamed: 0'],inplace=True)
df.head()

,coupon_id,customer_id,redemption_status,brand,brand_type,category,cd_sum,coupon_discount_x,coupon_used_x,item_counts,no_of_customers,od_sum,other_discount_x,price_sum,qu_sum,quantity_x,selling_price_x,t_counts,total_discount_mean,total_discount_sum,campaign_type,campaign_duration,age_range,marital_status,rented,family_size,no_of_children,income_bracket,coupon_discount_y,coupon_used_y,day,dow,no_of_items,month,other_discount_y,quantity_y,selling_price_y,cdd_sum,customer_id_count,odd_sum,qa_sum,pprice_sum
0,27,1053,0,1105,0,6,-5349.93,-1.241694,191.0,125,19.224000,-177447.090167,-44.367382,436274.446738,4661.0,1.169768,117.276864,3713.0,-45.609077,-182797.020167,0,47,3.0,1.0,0.0,1.0,0.0,5.0,-0.287258,1,11,2,208,5,-25.583099,340.487097,163.966826,-89.05,310,-7930.760842,105551,50829.715972
1,116,48,0,56,1,6,0.00,0.000000,0.0,3,20.333333,-586.140000,-10.207037,5323.510000,86.0,1.050926,70.885046,75.0,-10.207037,-586.140000,0,47,2.0,0.0,0.0,2.0,0.0,3.0,-3.215039,12,4,5,244,6,-20.013685,31.540260,188.638474,-1237.79,385,-7705.268702,12143,72625.812434
2,635,205,0,560,0,11,-605.54,-2.987321,15.0,67,1.761194,-12475.083333,-76.767366,48980.998333,202.0,1.353802,342.826195,142.0,-79.754687,-13080.623333,1,32,3.0,0.0,0.0,2.0,0.0,7.0,-2.212082,85,27,2,533,3,-12.873370,1.392784,112.067666,-2145.72,970,-12487.169143,1351,108705.636349
3,644,1050,0,611,0,6,-17.81,-4.452500,1.0,4,3.500000,-260.750000,-4.451964,3697.783333,47.0,1.110390,88.694903,39.0,-8.904464,-278.560000,0,47,3.0,0.0,0.0,2.0,0.0,5.0,-0.751477,2,9,4,216,8,-12.880868,1.291139,100.896997,-178.10,237,-3052.765833,306,23912.588333
4,1017,1489,0,1558,0,6,-569.92,-0.198055,23.0,32,27.312500,-46796.046667,-30.665116,289181.848333,1610.0,1.071044,200.991719,1420.0,-30.863170,-47365.966667,0,48,3.0,0.0,0.0,2.0,0.0,3.0,-0.471548,10,2,4,327,5,-12.264174,247.443060,85.016352,-265.01,562,-6892.466021,139063,47779.189609


In [7]:
x = df.drop(['redemption_status'],axis=1)
y = df.redemption_status

In [8]:
def cal_vif(x):
    thresh = 5
    output= pd.DataFrame()
    k = x.shape[1]
    vif = [variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
    for i in range(1,k):
        print('Iteration no ',i)
        print(vif)
        a = np.argmax(vif)
        print('Max vif is for variable no : ',a)
        if(vif[a]<=thresh):
            break
        if(i==1):
            output = x.drop(x.columns[a],axis=1)
            vif = [variance_inflation_factor(output.values,j) for j in range(output.shape[1])]
        elif(i>1):
            output = output.drop(output.columns[a],axis=1)
            vif = [variance_inflation_factor(output.values,j) for j in range(output.shape[1])]
    return(output)

In [9]:
final_fea = cal_vif(x)

Iteration no  1
[7.821118355371877, 4.072977408416887, 2.2777592986055546, 3.0664861809437918, 12.12848795713522, inf, inf, 101.97063421325727, 62.897562223825275, 3.757528002471387, inf, inf, 466.82044567716554, 1238.6720445201036, 23.815352752283374, 4.292680088944399, 1393.139228059251, inf, inf, 1.891231755617685, 65.46441411162117, 10.086637991085297, 11.850552406874884, 1.1376591756517584, 284.2514270663596, 42.161491655480575, 10.582370849152763, 5.360967584334791, 13.292161586450577, 4.335086482365526, 4.579390826507437, 35.97953937197585, 5.623737495934658, 48.312038191987746, 5.5327421198474465, 97.3843288704673, 21.52865218187338, 170.60109413414344, 61.965828788047546, 6.667942624334332, 130.233797304354]
Max vif is for variable no :  5
Iteration no  2
[7.821118355371877, 4.072977408416887, 2.2777592986055533, 3.066486180943794, 12.12848795713522, inf, 101.97063421326074, 62.897562223823954, 3.7575280024713855, 96291.53611089542, inf, 466.82044567716554, 1238.6720445201036,

Iteration no  13
[7.320605787612219, 3.994224196381967, 2.1231525108522975, 2.815520252170892, 10.91435262827439, 8.184373558637107, 3.0326760325402287, 3.503908207508685, 9.12795123145395, 2.6714344699820463, 15.04809626278921, 17.581487195430697, 3.9103136912482133, 1.4948117009616286, 8.923277291931159, 1.388280018926996, 1.1011723450335262, 1.380917132110334, 9.047996131485377, 4.654038040865127, 13.163472841955791, 4.214391404128683, 4.3955745696142055, 5.317743223024511, 14.434081009649987, 5.437147978135198, 20.037158000989358, 6.548743523310978, 7.041372287529066]
Max vif is for variable no :  26
Iteration no  14
[7.320485221048209, 3.9942221224177388, 2.1229504326389677, 2.815422690280868, 10.91435146495257, 8.184236345461132, 3.032672150455298, 3.503358515496246, 9.126634439358783, 2.6714302680359014, 15.046140040792398, 17.56610924935448, 3.9102224198889077, 1.4930667389660246, 8.923119097033014, 1.3857977137706798, 1.0989633174617888, 1.3796611041492064, 9.008537564202047, 

In [10]:
final_fea.head()

,customer_id,brand,brand_type,item_counts,no_of_customers,od_sum,other_discount_x,selling_price_x,campaign_type,marital_status,rented,no_of_children,coupon_discount_y,coupon_used_y,day,dow,month,quantity_y,pprice_sum
0,1053,1105,0,125,19.224000,-177447.090167,-44.367382,117.276864,0,1.0,0.0,0.0,-0.287258,1,11,2,5,340.487097,50829.715972
1,48,56,1,3,20.333333,-586.140000,-10.207037,70.885046,0,0.0,0.0,0.0,-3.215039,12,4,5,6,31.540260,72625.812434
2,205,560,0,67,1.761194,-12475.083333,-76.767366,342.826195,1,0.0,0.0,0.0,-2.212082,85,27,2,3,1.392784,108705.636349
3,1050,611,0,4,3.500000,-260.750000,-4.451964,88.694903,0,0.0,0.0,0.0,-0.751477,2,9,4,8,1.291139,23912.588333
4,1489,1558,0,32,27.312500,-46796.046667,-30.665116,200.991719,0,0.0,0.0,0.0,-0.471548,10,2,4,5,247.443060,47779.189609


In [11]:
x=final_fea
x.head()

,customer_id,brand,brand_type,item_counts,no_of_customers,od_sum,other_discount_x,selling_price_x,campaign_type,marital_status,rented,no_of_children,coupon_discount_y,coupon_used_y,day,dow,month,quantity_y,pprice_sum
0,1053,1105,0,125,19.224000,-177447.090167,-44.367382,117.276864,0,1.0,0.0,0.0,-0.287258,1,11,2,5,340.487097,50829.715972
1,48,56,1,3,20.333333,-586.140000,-10.207037,70.885046,0,0.0,0.0,0.0,-3.215039,12,4,5,6,31.540260,72625.812434
2,205,560,0,67,1.761194,-12475.083333,-76.767366,342.826195,1,0.0,0.0,0.0,-2.212082,85,27,2,3,1.392784,108705.636349
3,1050,611,0,4,3.500000,-260.750000,-4.451964,88.694903,0,0.0,0.0,0.0,-0.751477,2,9,4,8,1.291139,23912.588333
4,1489,1558,0,32,27.312500,-46796.046667,-30.665116,200.991719,0,0.0,0.0,0.0,-0.471548,10,2,4,5,247.443060,47779.189609


In [12]:
x.columns

Index(['customer_id', 'brand', 'brand_type', 'item_counts', 'no_of_customers',
       'od_sum', 'other_discount_x', 'selling_price_x', 'campaign_type',
       'marital_status', 'rented', 'no_of_children', 'coupon_discount_y',
       'coupon_used_y', 'day', 'dow', 'month', 'quantity_y', 'pprice_sum'],
      dtype='object')

In [13]:
from imblearn.over_sampling import SMOTE

# Standardizig the Data
col_names = ['item_counts', 'no_of_customers',
       'od_sum', 'other_discount_x','day', 'dow', 'month',
       'selling_price_x','no_of_children','coupon_discount_y', 'coupon_used_y',
        'quantity_y','pprice_sum']
features = x[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
x[col_names] = features


# setting up testing and training sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1990)

sm = SMOTE(random_state=1990, ratio=1.0)
x_train, y_train = sm.fit_sample(x_train, y_train)

In [14]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((108724, 19), (108724,), (23511, 19), (23511,))

# Logistic Regression

In [15]:
LR = LogisticRegression()
LR.fit(x_train,y_train)
y_pred_LR = LR.predict(x_test)
print(classification_report(y_test,y_pred_LR))

              precision    recall  f1-score   support

           0       1.00      0.82      0.90     23278
           1       0.04      0.77      0.08       233

    accuracy                           0.82     23511
   macro avg       0.52      0.80      0.49     23511
weighted avg       0.99      0.82      0.89     23511



In [16]:
print(roc_auc_score(y_test,y_pred_LR))
Model = ['Logistic Regression']
ROC_AUC_Accuracy = [roc_auc_score(y_test,y_pred_LR)]

0.7965267358116322


# kNN

## Caution --- Algorithm - 'brute' was crashing the system, hence removed it.

In [13]:
params = {
    
    'n_neighbors': range(1,5),
    'weights': ['uniform','distance'],
    'algorithm': ['ball_tree','kd_tree'],
    'p': [1,2,3]
}

knn = KNeighborsClassifier()

rs = RandomizedSearchCV(estimator=knn,n_jobs=-1,cv=3,param_distributions=params,scoring='recall')
rs.fit(x,y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=KNeighborsClassifier(algorithm='auto',
                                                  leaf_size=30,
                                                  metric='minkowski',
                                                  metric_params=None,
                                                  n_jobs=None, n_neighbors=5,
                                                  p=2, weights='uniform'),
                   iid='warn', n_iter=10, n_jobs=-1,
                   param_distributions={'algorithm': ['ball_tree', 'kd_tree'],
                                        'n_neighbors': range(1, 5),
                                        'p': [1, 2, 3],
                                        'weights': ['uniform', 'distance']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring='recall', verbose=0)

In [14]:
knn = KNeighborsClassifier(**rs.best_params_)
knn.fit(x_train,y_train)
y_pred_knn = knn.predict(x_test)
print(roc_auc_score(y_test,y_pred_knn))
Model.append('k-Nearest-Neighbours')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_knn))

0.5692587854877434


# Naive Bayes

In [17]:
nb = GaussianNB()
nb.fit(x_train,y_train)
y_pred_nb = nb.predict(x_test)
print(roc_auc_score(y_test,y_pred_nb))
Model.append('Naive Bayes')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_nb))

0.7487823054574176


In [18]:
print(classification_report(y_test,y_pred_nb))

              precision    recall  f1-score   support

           0       1.00      0.88      0.93     23278
           1       0.05      0.62      0.09       233

    accuracy                           0.87     23511
   macro avg       0.52      0.75      0.51     23511
weighted avg       0.99      0.87      0.92     23511



# Decision Tree

In [19]:
params = {
    
    'criterion':['gini','entropy'],
    'splitter':['best','random'],
    'max_depth':range(1,10),
    'max_leaf_nodes':range(2,10,1),
    'max_features':['auto','log2']
    
}

dt = DecisionTreeClassifier()

rs = RandomizedSearchCV(estimator=dt,n_jobs=-1,cv=3,param_distributions=params,scoring='recall')
rs.fit(x,y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=DecisionTreeClassifier(class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort=False,
                                                    random_state=None,
                                                    splitter='best')

In [20]:
dt = DecisionTreeClassifier(**rs.best_params_)
dt.fit(x_train,y_train)
y_pred_dt = dt.predict(x_test)
print(roc_auc_score(y_test,y_pred_dt))
Model.append('Decision Tree')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_dt))

0.7485030718462826


In [21]:
print(classification_report(y_test,y_pred_dt))

              precision    recall  f1-score   support

           0       1.00      0.87      0.93     23278
           1       0.05      0.62      0.09       233

    accuracy                           0.87     23511
   macro avg       0.52      0.75      0.51     23511
weighted avg       0.99      0.87      0.92     23511



In [22]:
Model,ROC_AUC_Accuracy

(['Logistic Regression', 'Naive Bayes', 'Decision Tree'],
 [0.7965267358116322, 0.7487823054574176, 0.7485030718462826])

# Random Forest

In [30]:
params = {
    
    'n_estimators':range(10,100,10),
    'criterion':['gini','entropy'],
    'max_depth':range(2,10,1),
    'max_leaf_nodes':range(2,10,1),
    'max_features':['auto','log2']
    
}

rf = RandomForestClassifier()

rs = RandomizedSearchCV(estimator=rf,param_distributions=params,cv=3,scoring='recall',n_jobs=-1)
rs.fit(x,y)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs=None

In [31]:
rf = RandomForestClassifier(**rs.best_params_)
rf.fit(x_train,y_train)
y_pred_rf = rf.predict(x_test)
print(roc_auc_score(y_test,y_pred_rf))
Model.append('Random Forest')
ROC_AUC_Accuracy.append(roc_auc_score(y_test,y_pred_rf))

0.8026399145687116


In [32]:
print(classification_report(y_test,y_pred_rf))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95     23278
           1       0.07      0.70      0.13       233

    accuracy                           0.91     23511
   macro avg       0.53      0.80      0.54     23511
weighted avg       0.99      0.91      0.94     23511



# Now using the Bagging Classifiers

In [35]:
LR_Bag = BaggingClassifier(base_estimator=LR,n_estimators=100,n_jobs=-1,random_state=1)
#knn_Bag = BaggingClassifier(base_estimator=knn,n_estimators=100,n_jobs=-1,random_state=1)
nb_Bag = BaggingClassifier(base_estimator=nb,n_estimators=100,n_jobs=-1,random_state=1)
dt_Bag = BaggingClassifier(base_estimator=dt,n_estimators=100,n_jobs=-1,random_state=1)

In [36]:
x = np.array(x)

In [37]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = LR_Bag
name = 'Bagged-LR'
#for model,name in zip([LR_Bag,knn_Bag,nb_Bag,dt_Bag],['Bagged-LR','Bagged-kNN','Bagged-NB','Bagged-DT']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Bagged-LR is 0.80 with variacne of (+/-) 0.00086


In [38]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = nb_Bag
name = 'Bagged-NB'
#for model,name in zip([LR_Bag,knn_Bag,nb_Bag,dt_Bag],['Bagged-LR','Bagged-kNN','Bagged-NB','Bagged-DT']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Bagged-NB is 0.75 with variacne of (+/-) 0.00030


In [39]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = dt_Bag
name = 'Bagged-DT'
#for model,name in zip([LR_Bag,knn_Bag,nb_Bag,dt_Bag],['Bagged-LR','Bagged-kNN','Bagged-NB','Bagged-DT']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Bagged-DT is 0.79 with variacne of (+/-) 0.00088


## Now Moving Towards Boosting?

In [40]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [43]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMModel,LGBMClassifier

In [44]:
LR_Boost = AdaBoostClassifier(base_estimator=LR,n_estimators=100,learning_rate=0.01,random_state=1)
#knn_Boost = AdaBoostClassifier(base_estimator=knn,n_estimators=100,learning_rate=0.01,random_state=1)
nb_Boost = AdaBoostClassifier(base_estimator=nb,n_estimators=100,learning_rate=0.01,random_state=1)
dt_Boost = AdaBoostClassifier(base_estimator=dt,n_estimators=100,learning_rate=0.01,random_state=1)
rf_Boost = AdaBoostClassifier(base_estimator=rf,n_estimators=100,learning_rate=0.01,random_state=1)
gb_Boost = GradientBoostingClassifier(n_estimators=100,learning_rate=0.01)
lgbm = LGBMClassifier(objective='binary',n_estimators=100,reg_alpha=2,reg_lambda=5,random_state=1,learning_rate=0.01,is_unbalance=True)

In [45]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = LR_Boost
name = 'Boosted-LR'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Boosted-LR is 0.73 with variacne of (+/-) 0.00053


In [46]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = nb_Boost
name = 'Boosted-NB'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Boosted-NB is 0.80 with variacne of (+/-) 0.00016


In [47]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = dt_Boost
name = 'Boosted-DT'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Boosted-DT is 0.79 with variacne of (+/-) 0.00057


In [51]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = rf_Boost
name = 'Boosted-Random Forest'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Boosted-Random Forest is 0.79 with variacne of (+/-) 0.00072


In [50]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = gb_Boost
name = 'Gradient Boost'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
Gradient Boost is 0.79 with variacne of (+/-) 0.00111


In [49]:
kf = KFold(n_splits=3,shuffle=True,random_state=1)
model = lgbm
name = 'LGBM'
#for model,name in zip([LR_Boost,nb_Boost,dt_Boost,rf_Boost,lgbm],['Boosted-LR','Boosted-NB','Boosted-DT','Boosted - Random Forest','LGBM']):
roc_acc = []
for train,test in kf.split(x,y):
    x_train = x[train,:]
    x_test = x[test,:]
    y_train = y[train]
    y_test = y[test]
    sm = SMOTE(random_state=1990, ratio=1.0)
    x_train, y_train = sm.fit_sample(x_train, y_train)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    score = roc_auc_score(y_test,y_pred)
    fpr,tpr,_ = roc_curve(y_test,y_pred)
    roc_acc.append(auc(fpr,tpr))
Model.append(name)
ROC_AUC_Accuracy.append(np.mean(roc_acc))
print('The AUC Score for')
print('%s is %0.02f with variacne of (+/-) %0.5f'%(name,np.mean(roc_acc),np.var(roc_acc,ddof=1)))

The AUC Score for
LGBM is 0.78 with variacne of (+/-) 0.00078


In [52]:
final_result = pd.DataFrame({'Model - SMOTE Data':Model,'Accuracy':ROC_AUC_Accuracy})
final_result

,Model - SMOTE Data,Accuracy
0,Logistic Regression,0.796527
1,Naive Bayes,0.748782
2,Decision Tree,0.748503
3,Random Forest,0.828211
4,Random Forest,0.816577
5,Random Forest,0.802640
6,Bagged-LR,0.804735
7,Bagged-NB,0.750271
8,Bagged-DT,0.787083
9,Boosted-LR,0.734306
